In [ ]:
import pandas as pd
import glob
from pyspark.sql.types import *

In [ ]:
files = glob.glob("/lakehouse/default/Files/*.xlsx")

In [ ]:
def combine_sheets(filename):
    workbook = pd.read_excel(filename, sheet_name=None)

    df_all_sheets = pd.DataFrame()

    for sheet_name, sheet in workbook.items():

        df_sheet = pd.melt(
            sheet,
            id_vars=["ProductKey"],
            var_name="Date",
            value_name="Qty"
        )

        df_sheet.dropna(subset=["Qty"], inplace=True)
        df_all_sheets = pd.concat([df_all_sheets, df_sheet], ignore_index=True)

    return(df_all_sheets)

In [ ]:
def combine_files(files):
    df_all_files = pd.DataFrame()

    for filename in files:
        file = combine_sheets(filename)
        df_all_files = pd.concat([df_all_files, file], ignore_index=True) \
            .sort_values(by=["Date"])

    return(df_all_files)

In [ ]:
df_all_files = combine_files(files)
print(df_all_files)

In [ ]:
schema = StructType() \
    .add("ProductKey", IntegerType(), True) \
    .add("Date", DateType(), True) \
    .add("Qty", IntegerType(), True)

In [ ]:
df_spark = spark.createDataFrame(df_all_files, schema=schema)
display(df_spark)

In [ ]:
df_spark.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("tb_combined")